In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import librosa
import librosa.display as ipd
import os
import warnings

warnings.filterwarnings('ignore')
import shutil
import kagglehub




In [48]:


file_names1 = [f for f in os.listdir('ses/ses_dosyalarim_tess/TESS Toronto emotional speech set data/YAF_angry')]
    
    
d = pd.DataFrame({
'Dosya' :file_names1,
})

print(d.head())






                Dosya
0  YAF_back_angry.wav
1   YAF_bar_angry.wav
2  YAF_base_angry.wav
3  YAF_bath_angry.wav
4  YAF_bean_angry.wav


In [ ]:
def ravdess():
    emotion_map = {
        '01': 'neutral',
        '02': 'calm',
        '03': 'happy',
        '04': 'sad',
        '05': 'angry',
        '06': 'fearful',
        '07': 'disgust',
        '08': 'surprised'
    }

    ravdess_path = 'ses/ses_dosyalarim_ravdes'

    #veri listeleri
    X = []  # mfcc 
    y = []  # Etiketler 

    # tüm wav dosyalarını gez
    for filename in os.listdir(ravdess_path):
        if filename.endswith('.wav'):
            file_path = os.path.join(ravdess_path, filename)
            
            emotion_code = filename.split('-')[2]
            emotion_label = emotion_map.get(emotion_code)

            y_audio, sr = librosa.load(file_path, sr=16000)
            y_audio, _ = librosa.effects.trim(y_audio)
            y_audio = librosa.util.normalize(y_audio)

            mfcc = librosa.feature.mfcc(y=y_audio, sr=sr, n_mfcc=20)

            if mfcc.shape[1] >= 100:
                mfcc = mfcc[:, :100]
            else:
                pad_width = 100 - mfcc.shape[1]
                mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')

            X.append(mfcc)
            y.append(emotion_label)

    # numpy dizisine çevir
    X = np.array(X)
    y = np.array(y)

    return X,y



In [ ]:
def tess():
    tess_path = 'ses/ses_dosyalarim_tess/TESS Toronto emotional speech set data'

    X1 = []
    y1 = []
    for filename in os.listdir(tess_path):
        folder_path = os.path.join(tess_path,filename)
        
        for file in os.listdir(folder_path):
            if file.endswith('.wav'):
                file_path = os.path.join(folder_path,file)
                emotion = file.split('_')[2].split('.')[0]

                
                
                y_audio, sr = librosa.load(file_path,sr=16000)
                y_audio, _ = librosa.effects.trim(y_audio)
                y_audio = librosa.util.normalize(y_audio)

                mfcc = librosa.feature.mfcc(y=y_audio, sr=sr, n_mfcc=20)
                if mfcc.shape[1] >= 100:
                    mfcc = mfcc[:, :100]
                else:
                    pad_width = 100 - mfcc.shape[1]
                    mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')

                X1.append(mfcc)
                y1.append(emotion)

    X1 = np.array(X1)
    y1 = np.array(y1)
    print(X1.shape)    
    return X1,y1




In [51]:
X,y = ravdess()
X1,y1 = tess()

result = np.concatenate([X,X1], axis=0)
result_y = np.concatenate([y,y1],axis=0)




(2800, 20, 100)


In [ ]:
print(result.shape)
print(result_y.shape)


(4240, 20, 100)
(4240,)


In [ ]:


from joblib import dump
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


X_flat = result.reshape(result.shape[0], -1)
print(X_flat.shape)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(result_y)


pca = PCA(n_components=0.95)  
X_reduced = pca.fit_transform(X_flat)

print("pca sonrası şekil:", X_reduced.shape)

#eğitim ve test setine ayır
X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y_encoded, test_size=0.2, random_state=42
)



print(y_train.shape)
print(y_test.shape)


dump(pca, "pca_transform.joblib")
dump(label_encoder, "label_encoder.joblib")


(4240, 2000)
[6 6 6 ... 8 8 8]
pca sonrası şekil: (4240, 137)
(3392,)
(848,)


['label_encoder.joblib']

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier


param_grid = {
    'hidden_layer_sizes': [(128,), (128, 64)],
    'activation': ['relu', 'tanh'],
    'learning_rate_init': [0.001, 0.01],
    'max_iter': [200, 300]
}

mlp = MLPClassifier(random_state=35)
grid_search = GridSearchCV(mlp, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("En iyi parametreler:", grid_search.best_params_)

En iyi parametreler: {'activation': 'relu', 'hidden_layer_sizes': (128,), 'learning_rate_init': 0.01, 'max_iter': 200}


In [ ]:
from joblib import dump

best_params = grid_search.best_params_
mlp = MLPClassifier(
    early_stopping=True,
    hidden_layer_sizes=best_params['hidden_layer_sizes'],
    activation=best_params['activation'],
    learning_rate_init=best_params['learning_rate_init'],
    max_iter=best_params['max_iter'],
    
)

mlp.fit(X_train, y_train)


,hidden_layer_sizes,"(128,)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.01
,power_t,0.5
,max_iter,200
,shuffle,True
,random_state,None


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


y_pred = mlp.predict(X_test)

print(y_pred)
print("Doğruluk Oranı:", accuracy_score(y_test, y_pred))
print("Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Doğruluk Oranı: 0.7841981132075472

Sınıflandırma Raporu:

              precision    recall  f1-score   support

       angry       0.84      0.85      0.84       118
        calm       0.38      0.32      0.34        38
     disgust       0.77      0.87      0.82       118
        fear       0.97      0.97      0.97        80
     fearful       0.41      0.31      0.35        39
       happy       0.77      0.81      0.79       119
     neutral       0.73      0.81      0.77        99
          ps       0.96      0.91      0.94        80
         sad       0.82      0.79      0.81       118
   surprised       0.55      0.46      0.50        39

    accuracy                           0.78       848
   macro avg       0.72      0.71      0.71       848
weighted avg       0.78      0.78      0.78       848



In [ ]:
from sklearn.model_selection import StratifiedKFold


kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in kf.split(X_reduced, y_encoded):
    X_train, X_test = X_reduced[train_index], X_reduced[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    model = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', max_iter=200, learning_rate_init=0.01)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print("Tüm fold'ların doğrulukları:", accuracies)
print("Ortalama doğruluk:", np.mean(accuracies))

Tüm fold'ların doğrulukları: [0.7535377358490566, 0.7653301886792453, 0.7841981132075472, 0.7735849056603774, 0.7806603773584906]
Ortalama doğruluk: 0.7714622641509434
